In [94]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
plt.ion()
InteractiveShell.ast_node_interactivity='all'

In [104]:
train_df=pd.read_csv('./spambasetrain.csv',header=None)
test_df=pd.read_csv('./spambasetest.csv',header=None)
name={i:'x'+str(i) for i in range(0,9)}
name[9]='y'
train_df.rename(columns=name,inplace=True)
test_df.rename(columns=name,inplace=True)
train=train_df.convert_objects(convert_numeric=True)
train.head(10)
# train,test=pd.to_numeric(train_df),pd.to_numeric(test_df)
train.dtypes

/Users/joaintang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys


,x0,x1,x2,x3,x4,x5,x6,x7,x8,y
0,0.000,0.316,0.0,0.000,0.000,0.000,1.125,2,9,0
1,0.000,0.735,0.0,0.735,0.000,0.000,2.571,10,18,0
2,0.000,0.105,0.0,0.000,0.158,0.000,1.494,10,139,1
3,0.000,0.299,0.0,0.000,0.000,0.199,2.465,28,106,1
4,0.000,0.000,0.0,0.817,0.000,0.000,1.857,15,39,0
5,0.000,0.000,0.0,2.272,0.000,0.000,1.750,7,14,0
6,0.055,0.334,0.0,0.055,0.000,0.055,1.685,6,59,0
7,0.000,0.162,0.0,0.000,0.000,0.000,2.643,34,193,0
8,0.000,0.075,0.0,0.613,0.532,0.137,7.300,763,2453,1
9,0.000,0.052,0.0,0.078,0.000,0.235,3.153,121,618,0


x0    float64
x1    float64
x2    float64
x3    float64
x4    float64
x5    float64
x6    float64
x7      int64
x8      int64
y       int64
dtype: object

In [92]:
def P_C(df):
    return sum(df['y'])/len(df['y'])

In [93]:
Pc_test,Pc_train=P_C(test_df),P_C(train_df)
Pc_test,Pc_train

(0.41, 0.4018006002000667)

In [102]:
def calc_gaussian_param(df,y=0):
    res=np.zeros((df.shape[1],2))
    for i,xi in enumerate(df.columns):
        res[i][0]=df.query("y=="+str(y))[xi].mean()
        res[i][1]=df.query("y=="+str(y))[xi].var()

    return pd.DataFrame(data=res,columns=['miu','sigma'])

In [108]:
params=calc_gaussian_param(train_df,0),calc_gaussian_param(train_df,1)
params[0]
params[1]

,miu,sigma
0,0.048426,0.088306
1,0.157487,0.070180
2,0.016960,0.004845
3,0.110713,0.417588
4,0.011503,0.004080
5,0.025892,0.089269
6,2.376590,37.447271
7,18.182274,1915.266813
8,164.143255,144134.168535
9,0.000000,0.000000


,miu,sigma
0,0.020247,0.007542
1,0.103439,0.032143
2,0.007847,0.002416
3,0.491662,0.547792
4,0.173151,0.129976
5,0.084901,0.527283
6,9.496347,2965.984486
7,97.209129,36369.991113
8,453.512863,614182.120474
9,1.000000,0.000000


In [46]:
gauss_param_train_0.dtypes

miu      float64
sigma    float64
dtype: object

In [50]:
def gaussian(x,miu,sigma2):

    def fun(x):
        return np.exp(-(x-miu)**2/(2*sigma2))/np.sqrt(2*np.pi*sigma2)
    return fun(x)

In [51]:
def mult_gauss(xs,params,inds=list(range(9))):
    res=1
    for i in inds:
        m,s=params.iloc[i]
        res*=gaussian(xs[i],m,s)
    return res

In [73]:
s=0
for i,d in train.iterrows():
    px_c0=mult_gauss(d,params[0])*(1-Pc_train)
    px_c1=mult_gauss(d,params[1])*Pc_train
#     print("P(X|c=0):{},P(X|c=1):{},label:{}".format(np.log(px_c0),np.log(px_c1),train['y'].iloc[i]))
    if px_c0>px_c1 and train['y'].iloc[i]==0 or px_c0<=px_c1 and train['y'].iloc[i]==1:
        s+=1
print(s/train.shape[0])

0.731243747915972


In [106]:
test=test_df.convert_objects(convert_numeric=True)
feats=[
    [1,2,3,4,7]
]
for feat in feats:
    s=0
    for i,d in test.iterrows():
        px_c0=mult_gauss(d,params[0],inds=feat)*(1-Pc_train)
        px_c1=mult_gauss(d,params[1],inds=feat)*Pc_train
    #     print("P(X|c=0):{},P(X|c=1):{},label:{}".format(np.log(px_c0),np.log(px_c1),train['y'].iloc[i]))
        if px_c0>=px_c1 and test['y'].iloc[i]==0 or px_c0<px_c1 and test['y'].iloc[i]==1:
            s+=1
    print(feat,s/test.shape[0],s,test.shape[0]-s)

[1, 2, 3, 4, 7] 0.815 163 37


/Users/joaintang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
